In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from matplotlib import pyplot as plt
from mtcnn import MTCNN

In [ ]:
from utils import count_file_and_folder

folder_count, total_file_count = count_file_and_folder('./data1')
print(f"\nNumber of folders: {folder_count}")
print(f"Total number of files: {total_file_count}")

## Preprocessing

In [ ]:
from utils import reset_folder

reset_folder('./detected_faces')
print("Contents of detected_faces cleared.")

In [ ]:
from services.preprocessing import Preprocessing

image_paths = Preprocessing.get_image_paths('./data1')
Preprocessing.detect_and_save(image_paths, './detected_faces')

In [ ]:
from utils import count_file_and_folder

folder_count, total_file_count = count_file_and_folder('./detected_faces')
print(f"\nNumber of folders: {folder_count}")
print(f"Total number of files: {total_file_count}")

## Extracting embeddings

In [ ]:
import os

reset_folder('./extracted_embeddings')
print("Contents of extracted_embeddings cleared.")

In [ ]:
from facenet_pytorch import InceptionResnetV1
import torch
from services.face_embedding import FaceEmbedding

input_directory = './detected_faces'
output_directory = './extracted_embeddings'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
face_embedder = InceptionResnetV1(pretrained='vggface2').eval().to(device)

for folder_name in os.listdir(input_directory):
  folder_path = os.path.join(input_directory, folder_name)

  # Create subfolder in output_directory for the current label
  label_output_directory = os.path.join(output_directory, folder_name)
  os.makedirs(label_output_directory, exist_ok=True)

  # Extract embeddings from the current folder with data augmentation
  label, embeddings = FaceEmbedding.images_embedding(folder_path, face_embedder, device, label_output_directory)

print("Embeddings extraction complete.")


In [ ]:
from utils import count_file_and_folder

folder_count, total_file_count = count_file_and_folder('./extracted_embeddings')
print(f"\nNumber of folders: {folder_count}")
print(f"Total number of files: {total_file_count}")

RECOGNIZING FACES - Classification model - Multilayered Neural Network

In [ ]:
from services.face_model import get_train_data

data_dir = "./extracted_embeddings"
embeddings, labels = get_train_data(data_dir)
print(len(embeddings))
print(len(labels))

In [ ]:
import json
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from services.face_model import FaceModel, get_train_data

torch.cuda.is_available = lambda : False
device = torch.device("cpu")
data_dir = "./extracted_embeddings"
embeddings, labels = get_train_data(data_dir)
storage_label = labels

X = torch.tensor(embeddings, dtype=torch.float32)
print("x", type(X))
label_encoder = LabelEncoder()
y = torch.tensor(label_encoder.fit_transform(labels), dtype=torch.long)

# Split the data into 80% training, 10% validation, and 10% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize the model
input_size = X_train.shape[1]
num_classes = len(set(y_train))
model = FaceModel(input_size, num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(X_train), batch_size):
        inputs = X_train[i:i+batch_size].to(device)
        labels = y_train[i:i+batch_size].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_inputs = X_val.to(device)
        val_labels = y_val.to(device)

        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

model.eval()
with torch.no_grad():
    test_inputs = X_test.to(device)
    test_labels = y_test.to(device)

    test_outputs = model(test_inputs)
    test_loss = criterion(test_outputs, test_labels)

predicted_labels = label_encoder.inverse_transform(torch.argmax(test_outputs, dim=1).cpu().numpy())
print("test_outputs", len(test_outputs), len(predicted_labels))
true_labels = label_encoder.inverse_transform(y_test.cpu().numpy())

print("Classification Report: ")
torch.save(model.state_dict(), "./weights")
file = open('metadata.json', 'w', encoding='utf-8')
json.dump({"input_size": input_size, "num_classes": num_classes, "labels": list(storage_label)}, file)
file.close()

In [ ]:
import cv2
import json
import torch
from mtcnn import MTCNN
from services.face_model import FaceModel
from facenet_pytorch import InceptionResnetV1
from sklearn.preprocessing import LabelEncoder
from services.face_embedding import FaceEmbedding
from services.face_detection import FaceDetection

file = open('metadata.json')
data = json.load(file)
file.close()
model = FaceModel(data['input_size'], data['num_classes'])
model.load_state_dict(torch.load('weights'))
model.eval()

label_encoder = LabelEncoder()
label_encoder.fit_transform(data['labels'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
face_embedder = InceptionResnetV1(pretrained='vggface2').eval().to(device)

face_detector = MTCNN()
# rgb_img, faces = FaceDetection.detect('./images/selena-gomez.jpeg', face_detector)
# img = cv2.imread('./images/selena-gomez.jpeg')
# x, y, w, h = faces[0]["box"]
# face_roi = img[y: y + h, x: x + w]
# resized_face = cv2.resize(face_roi, (224, 224))
# cv2.imwrite("./images/abc.jpg", resized_face)
# FaceDetection.draw("123", rgb_img, faces)

embedding = FaceEmbedding.image_embedding('./images/abc.jpg', face_embedder, device)
torch_embedding = torch.tensor([embedding], dtype=torch.float32)
labels = model(torch_embedding.to(device))
predicted_labels = label_encoder.inverse_transform(torch.argmax(labels, dim=1).cpu().numpy())
print("test_outputs", len(labels), len(predicted_labels))
print("predicted_labels", predicted_labels)